In [38]:
!python -m jupytools export -nb "xx_extract_features.ipynb" -o ..

Exported: xx_extract_features.ipynb -> ../extract_features.py
1 notebook(s) exported into folder: ..


In [6]:
import jupytools.syspath
jupytools.syspath.add('..')

In [7]:
#export
import json
from collections import OrderedDict
from multiprocessing import cpu_count
import pandas as pd
from tqdm.auto import tqdm
from utils import parallel, parallel_chain

## Event Data Fields

In [8]:
#export
DEFAULT_EVENT_FEATUERS = (
    'game_time', 'coordinates.y', 'coordinates.stage_height',
    'coordinates.stage_width', 'coordinates.x',
    'description', 'media_type', 'identifier',
    'duration', 'total_duration'
)

In [9]:
#export
class EventParser:
    def __init__(self, keys):
        self.keys = keys
    def __call__(self, json_str):
        obj = json.loads(json_str)
        obj = pd.io.json.json_normalize(obj)
        obj = obj.T[0].to_dict()
        row = {k: obj.get(k) for k in self.keys}
        return row

In [10]:
#export
def fillna(df, column, method='mean', value=None, fallback=0):
    if df[column].isna().all():
        value = fallback
    elif method == 'mean':
        value = df[column].mean()
    elif method == 'mode':
        value = df[column].value_counts().index[0]
    elif method == 'const':
        assert value is not None
    else:
        raise RuntimeError(f'invalid imputing method: {method}')
    df[column] = df[column].fillna(value)
    return df

In [11]:
#export
def extract_event_data(df, features=DEFAULT_EVENT_FEATUERS,
                       num_workers=cpu_count(), pbar=False,
                       **opts):
    
    parse_row = EventParser(features)
    event_data = df.event_data
    if pbar:
        event_data = tqdm(event_data, desc='Processing events')
    df = pd.DataFrame(parallel(parse_row, event_data, num_workers))
    df = fillna(df, 'game_time', method='mean')
    df = fillna(df, 'coordinates.x', method='mode')
    df = fillna(df, 'coordinates.y', method='mode')
    df = fillna(df, 'coordinates.stage_height', method='mode')
    df = fillna(df, 'coordinates.stage_width', method='mode')
    df = fillna(df, 'description', method='mode', fallback='none')
    df = fillna(df, 'media_type', method='const', value='none')
    df = fillna(df, 'identifier', method='const', value='none')
    df = fillna(df, 'duration', method='mean')
    df = fillna(df, 'total_duration', method='mean')
    return df

In [12]:
#export
def extend_with_event_data(dataset, **opts):
    events = extract_event_data(dataset, **opts)
    joined = pd.concat([
        dataset.drop(columns='event_data'),
        events.drop(columns='game_time')
    ], axis=1)
    return joined

## Baseline Features

In [13]:
#export
def basic_features(group):
    row = OrderedDict()
    row['game_session'] = group.game_session.iloc[-1]
    row['installation_id'] = group.installation_id.iloc[-1]
    row['events_count'] = len(group)
    row['game_session_uniq'] = len(group.game_session.unique())
    row['title_uniq'] = len(group.title.unique())
    return row

In [14]:
#export
def event_data_features(group):
    event_cnt = group.event_code.value_counts()
    row = OrderedDict()
    row['event_count_max'] = group.event_count.max()
    row['event_code_least'] = event_cnt.index[-1]
    row['event_code_most'] = event_cnt.index[0]
    row['event_code_uniq'] = len(event_cnt)
    return row

In [15]:
#export
def game_time_features(group):
    row = OrderedDict()
    gt = group.game_time
    row['game_time_min'] = gt.min()
    row['game_time_max'] = gt.max()
    row['game_time_mean'] = gt.mean()
    row['game_time_std'] = gt.std()
    row['game_time_skew'] = gt.skew()
    row['game_time_sum'] = gt.sum()
    return row

In [16]:
#export
def type_features(group):
    n = len(group)
    type_cnt = group.type.value_counts()
    row = OrderedDict()
    row['type_uniq'] = len(type_cnt)
    row['type_least'] = type_cnt.index[-1]
    row['type_most'] = type_cnt.index[0]
    row['type_clip_count'] = type_cnt.get('Clip', 0)
    row['type_activity_count'] = type_cnt.get('Activity', 0)
    row['type_game_count'] = type_cnt.get('Game', 0)
    row['type_assessment_count'] = type_cnt.get('Assessment', 0)
    row['type_clip_freq'] = row['type_clip_count']/n
    row['type_activity_freq'] = row['type_activity_count']/n
    row['type_game_freq'] = row['type_game_count']/n
    row['type_assessment_freq'] = row['type_assessment_count']/n
    return row

In [17]:
#export
def world_features(group):
    n = len(group)
    world_cnt = group.world.value_counts()
    row = OrderedDict()
    row['world_uniq'] = len(world_cnt)
    row['world_least'] = world_cnt.index[-1]
    row['world_most'] = world_cnt.index[0]
    row['world_none_count'] = world_cnt.get('NONE', 0)
    row['world_magmapeak_count'] = world_cnt.get('MAGMAPEAK', 0)
    row['world_treetopcity_count'] = world_cnt.get('TREETOPCITY', 0)
    row['world_crystalcaves_count'] = world_cnt.get('CRYSTALCAVES', 0)
    row['world_none_freq'] = row['world_none_count']/n
    row['world_magmapeak_freq'] = row['world_magmapeak_count']/n
    row['world_treetopcity_freq'] = row['world_treetopcity_count']/n
    row['world_crystalcaves_freq'] = row['world_crystalcaves_count']/n
    return row

In [18]:
#export
def media_features(group):
    n = len(group)
    media_cnt = group.media_type.value_counts()
    row = OrderedDict()
    row['media_uniq'] = len(media_cnt)
    row['media_least'] = media_cnt.index[-1]
    row['media_most'] = media_cnt.index[0]
    row['media_none_count'] = media_cnt.get('none', 0)
    row['media_audio_count'] = media_cnt.get('audio', 0)
    row['media_animation_count'] = media_cnt.get('animation', 0)
    row['media_none_freq'] = row['media_none_count']/n
    row['media_audio_freq'] = row['media_audio_count']/n
    row['media_animation_freq'] = row['media_animation_count']/n
    return row

In [19]:
#export
def timestamp_features(group):
    ts = group['timestamp']
    row = OrderedDict()
    
    row['ts_month_first'] = ts.dt.month.min()
    row['ts_month_last'] = ts.dt.month.max()
    row['ts_month_mean'] = ts.dt.month.mean()
    row['ts_month_std'] = ts.dt.month.std()
    row['ts_month_diff'] = row['ts_month_last'] - row['ts_month_first']
    row['ts_month_uniq'] = len(ts.dt.month.unique())
    
    row['ts_day_first'] = ts.dt.day.min()
    row['ts_day_last'] = ts.dt.day.max()
    row['ts_day_mean'] = ts.dt.day.mean()
    row['ts_day_std'] = ts.dt.day.std()
    row['ts_day_diff'] = row['ts_day_last'] - row['ts_day_first']
    row['ts_day_uniq'] = len(ts.dt.month.unique())
    
    dow_cnt = ts.dt.dayofweek.value_counts()
    row['ts_dow_least'] = dow_cnt.index[-1]
    row['ts_dow_most'] = dow_cnt.index[0]
    row['ts_dow_mean'] = ts.dt.dayofweek.mean()
    row['ts_dow_std'] = ts.dt.dayofweek.std()
    
    hour_cnt = ts.dt.hour.value_counts()
    row['ts_hour_least'] = hour_cnt.index[-1]
    row['ts_hour_most'] = hour_cnt.index[0]
    row['ts_hour_mean'] = ts.dt.hour.mean()
    row['ts_hour_std'] = ts.dt.hour.std()
    
    delta = ts.max() - ts.min()
    row['ts_delta_seconds'] = delta.total_seconds()
    return row

In [37]:
#export
SMOKE_TEST_FEATURES = [
    basic_features,
    event_data_features,
    game_time_features,
    type_features,
    world_features,
    media_features
]

class FeaturesExtractor:
    def __call__(self, dataset, *args, **kwargs):
        return self.extract(dataset, *args, **kwargs)
    def extract(self, dataset):
        raise NotImplemetedError()
        
class AssessmentFeatures(FeaturesExtractor):
    def __init__(self, steps):
        super().__init__()
        self.steps = steps
    def extract(self, group):
        g = group.reset_index(drop=True)
        assessments = g.index[g.title.str.contains('\(Assessment\)')].tolist()
        rows = []
        for i, line_no in enumerate(assessments):
            asm = g.iloc[:line_no+1].copy()
            row = OrderedDict()
            for step in self.steps:
                row.update(step(asm))
            rows.append(row)
        return rows
    
class BaselineFeatures(AssessmentFeatures):
    def __init__(self):
        super().__init__(steps=SMOKE_TEST_FEATURES)
        
class BaselineWithTimestampFeatures(AssessmentFeatures):
    def __init__(self):
        super().__init__(steps=SMOKE_TEST_FEATURES + [timestamp_features])

In [21]:
#export
def baseline_features(group):
    g = group.reset_index(drop=True)
    
    assessments = g.index[g.title.str.contains('\(Assessment\)')].tolist()
    
    steps = [
        basic_features,
        event_data_features,
        game_time_features,
        type_features,
        world_features,
        media_features
    ]
    
    rows = []
    for i, line_no in enumerate(assessments):
        asm = g.iloc[:line_no+1].copy()
        row = OrderedDict()
        for step in steps:
            row.update(step(asm))
        rows.append(row)
        
    return rows

## Generic Extractor Method

In [29]:
#export
def prepare_with_groups(groups, prepare_fn, num_workers=cpu_count(), targets=None, **opts):
    dataset = pd.DataFrame(parallel_chain(prepare_fn, groups, num_workers))
    dataset = dataset.reset_index(drop=True)
    if targets is not None:
        dataset = dataset[dataset.game_session.isin(targets.game_session)]
    return dataset    

In [23]:
#export
def prepare_with_events(dataset, prepare_fn, pbar=False, **opts):
    grouped = dataset.groupby('installation_id')
    total = grouped.ngroups
    groups = (g for _, g in grouped)
    if pbar:
        groups = tqdm(groups, total=total, desc='Preparing group') 
    return prepare_with_groups(groups, prepare_fn, **opts)    

In [24]:
#export
def prepare(dataset, prepare_fn, **opts):
    dataset = extend_with_event_data(dataset, **opts)
    dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
    return prepare_with_events(dataset, prepare_fn, **opts)